# covid-nsw
Driven by SQL, I explore [NSW COVID-19 cases by location](https://data.nsw.gov.au/search/dataset/ds-nsw-ckan-aefcde60-3b0c-4bc0-9af1-6fe652944ec2/details?q=).

## (0) Setup
Here, I load libraries and import the key input file into a SQLite table.

In [1]:
from glob import glob
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns

engine = create_engine('sqlite://', echo=False)

covid_df = pd.read_csv(glob('./input/confirmed_cases*.csv')[0])
covid_df.to_sql('covid', 
                engine, 
                if_exists='replace', 
                index=False)
covid_df.head(3)

,notification_date,postcode,lhd_2010_code,lhd_2010_name,lga_code19,lga_name19,confirmed_by_pcr,confirmed_cases_count
0,2020-01-25,2071,X760,Northern Sydney,14500,Ku-ring-gai (A),NaN,1
1,2020-01-25,2121,X760,Northern Sydney,16260,Parramatta (C),NaN,1
2,2020-01-25,2134,X700,Sydney,11300,Burwood (A),NaN,1


## (1) Cleaning

### Missing
There are missing values in both `postcode` and `confirmed_by_pcr` columns:

In [2]:
pd.read_sql_query("""
    SELECT SUM(CASE WHEN notification_date IS NULL THEN 1 ELSE 0 END) AS 'nd_miss',
        SUM(CASE WHEN postcode IS 'None' THEN 1 ELSE 0 END) AS 'pc_miss',
        SUM(CASE WHEN lhd_2010_code IS  NULL THEN 1 ELSE 0 END) AS 'lhdc_miss',
        SUM(CASE WHEN lhd_2010_name IS NULL THEN 1 ELSE 0 END) AS 'lhdn_miss',
        SUM(CASE WHEN lga_code19 IS NULL THEN 1 ELSE 0 END) AS 'lgac_miss',
        SUM(CASE WHEN lga_name19 IS NULL THEN 1 ELSE 0 END) AS 'lgan_miss',
        SUM(CASE WHEN confirmed_by_pcr IS NULL THEN 1 ELSE 0 END) AS 'pcr_miss',
        SUM(CASE WHEN confirmed_cases_count IS NULL THEN 1 ELSE 0 END) AS 'confirmed_cases_miss'
    FROM covid
    """, engine)

,nd_miss,pc_miss,lhdc_miss,lhdn_miss,lgac_miss,lgan_miss,pcr_miss,confirmed_cases_miss
0,0,530,0,0,0,0,41075,0


Although Local Government Area (LGA) information is not missing for rows with missing `postcode`, `postcode` is nevertheless difficult to impute as there can be multiple postcodes in an LGA (1:M).

Similarly, `confirmed_by_pcr` is difficult to impute. It records whether a case detected by Rapid Antigen Tests (RAT), reported to NSW Health, was also confirmed by PCR. All missing values were prior to 19 Jan 2022 and would be difficult to impute.

In [3]:
pd.read_sql_query("""
    SELECT MAX(notification_date) AS 'latest_pcr_miss_date'
    FROM covid
    WHERE confirmed_by_pcr IS NULL
    """, engine)

,latest_pcr_miss_date
0,2022-01-19


## (2) Analysis
### Daily Cases

In [9]:
daily_cases =\
    pd.read_sql_query("""
        SELECT date(notification_date) AS 'date',
            SUM(confirmed_cases_count) AS 'cases'
        FROM covid
        GROUP BY 1
        ORDER BY 1
        """, engine)

daily_cases

,date,cases
0,2020-01-25,3
1,2020-01-27,1
2,2020-03-01,2
3,2020-03-02,3
4,2020-03-03,6
...,...,...
810,2022-07-14,11621
811,2022-07-15,10933
812,2022-07-16,9878
813,2022-07-17,9871


In [5]:
# simple visualisation for this would go a long way!

pd.read_sql_query("""
    SELECT strftime('%m', notification_date) AS 'month',
        strftime('%Y', notification_date) AS 'year',
        SUM(confirmed_cases_count) AS 'cases'
    FROM covid
    GROUP BY 1, 2
    ORDER BY 2, 1
    """, engine)

,month,year,cases
0,01,2020,4
1,03,2020,2106
2,04,2020,794
3,05,2020,52
4,06,2020,88
5,07,2020,319
6,08,2020,271
7,09,2020,127
8,10,2020,149
9,11,2020,87


In [6]:
#### continue editing from here

### Daily Count
#### Total daily reported cases
As the table has each row representing a notified COVID case, we will aggregate them by `notification_date`:

In [7]:
pd.read_sql_query("""SELECT notification_date,
                         COUNT(*) cases
                     FROM covid_nsw_2
                     GROUP BY 1
                     ORDER BY 1""", engine)

OperationalError: (sqlite3.OperationalError) no such table: covid_nsw_2
[SQL: SELECT notification_date,
                         COUNT(*) cases
                     FROM covid_nsw_2
                     GROUP BY 1
                     ORDER BY 1]
(Background on this error at: http://sqlalche.me/e/14/e3q8)

#### Highest reported daily cases - top 10

In [ ]:
pd.read_sql_query("""SELECT notification_date date,
                         COUNT(*) cases
                     FROM covid_nsw_2
                     GROUP BY 1
                     ORDER BY 2 DESC
                     LIMIT 10""", engine)

#### "Zero" days
As the `covid_nsw_2` data only reports cases that have been notified, we will need to:
1. Generate the dates between the start and end of the reporting period. This will update automatically as we load in a new CSV from Data.NSW.
2. `LEFT JOIN` those dates with `covid_nsw_2` and find dates that were *not* in the `covid_nsw_2` table.

In [ ]:
pd.read_sql_query("""WITH RECURSIVE dates(date) AS (
                         VALUES((
                             SELECT MIN(notification_date)
                             FROM covid_nsw_2
                             )
                         )
                         UNION ALL
                         SELECT DATE(date, '+1 day')
                         FROM dates
                         WHERE date < (
                             SELECT MAX(notification_date)
                             FROM covid_nsw_2
                             )
                     )
                     
                     SELECT d.date zero_days
                     FROM dates d
                     LEFT JOIN covid_nsw_2 c
                         ON d.date = c.notification_date
                     WHERE notification_date IS NULL
                     ORDER BY 1""", engine)

### Geographic

#### "Greater Sydney"
The original dataset reports on *all* LGAs in NSW. In order to limit by "Greater Sydney", where needed, we will prepare a table that lists Greater Sydney LGAs.

Although there are some definition differences, we use the "Greater Sydney" LGAs as exhaustively defined in the COVID rules, published by the [NSW Government](https://www.nsw.gov.au/covid-19/stay-safe/protecting/advice-high-risk-groups/disability/local-councils-greater-sydney) and referenced in press conferences and media updates.

##### Importing, inspecting & cleaning
We will import & inspect a CSV that lists Greater Sydney LGAs, published by [NSW Government](https://www.nsw.gov.au/covid-19/stay-safe/protecting/advice-high-risk-groups/disability/local-councils-greater-sydney).

In [ ]:
greater_sydney_lgas_df = pd.read_csv('input/greater_sydney_lgas.csv')

greater_sydney_lgas_df.to_sql('greater_sydney_lgas', engine, if_exists='replace')

greater_sydney_lgas_df

The LGA names in `greater_sydney_lgas` need to align with our main table, `covid_nsw_2`.

We will need to `LEFT JOIN` the two tables and identify LGAs in `greater_sydney_lgas` but not in `covid_nsw_2`. These will be the LGAs in Greater Sydney that will need to have their names amended to match those in `covid_nsw_2`.

In [ ]:
pd.read_sql_query("""SELECT DISTINCT lga_name
                     FROM greater_sydney_lgas g
                     LEFT JOIN covid_nsw_2 c
                         ON g.lga_name = c.lga
                     WHERE lga IS NULL""", engine)

Finding the way that `covid_nsw_2` refers to the identified Greater Sydney LGAs above:

In [ ]:
pd.read_sql_query("""SELECT DISTINCT lga 
                     FROM covid_nsw_2
                     WHERE lga LIKE 'Hunter%'
                         OR lga LIKE '%Hills%'
                     ORDER BY 1 DESC""", engine)

Updating the `greater_sydney_lgas` table to ensure it has the names in `covid_nsw_2`:

*NB: with `sqlalchemy`, both `UPDATE` and `CREATE TABLE` produce 'ResourceClosedError: This result object does not return rows. It has been closed automatically.' ([Source](https://github.com/sqlalchemy/sqlalchemy/issues/5433)). Thus, instead, we are using `UNION` so that later analysis can use `IN` for this table.*

In [ ]:
greater_sydney_lgas_df_2 = pd.read_sql_query("""SELECT lga_name
                                             FROM greater_sydney_lgas
                                             UNION
                                             SELECT DISTINCT lga 
                                             FROM covid_nsw_2
                                             WHERE lga LIKE 'Hunter%'
                                                 OR lga LIKE '%Hills%'
                                             ORDER BY 1""", engine)

greater_sydney_lgas_df_2.to_sql('greater_sydney_lgas', engine, if_exists='replace', index=False)

pd.read_sql_query("""SELECT *
                     FROM greater_sydney_lgas""", engine)

#### LGAs
##### NSW - Top 10

In [ ]:
pd.read_sql_query("""SELECT lga,
                         COUNT(*) cases
                     FROM covid_nsw_2
                     GROUP BY 1
                     ORDER BY 2 DESC
                     LIMIT 10""", engine)

#####  Greater Sydney - ranked

In [ ]:
pd.read_sql_query("""SELECT lga,
                         COUNT(*) cases
                     FROM covid_nsw_2
                     WHERE lga IN (
                         SELECT *
                         FROM greater_sydney_lgas
                     )
                     GROUP BY 1
                     ORDER BY 2 DESC""", engine)

##### Total reported cases, during "Delta" - top 10 in NSW - compared with pre-"Delta" total cases
Although unclear of the exact date, the "Delta" wave (also known as Australia's 3rd wave, and NSW's 2nd wave), this likely began on or around 18 June 2021.

Source 1: [ABC News 2021a, 'Outbreak, How Australia lost control of the COVID Delta variant | ABC News'](https://www.youtube.com/watch?v=M9MPXCpyCnY&ab_channel=ABCNewsIn-depth)

Source 2: [ABC News 2021b, 'NSW Health records additional COVID-19 case, masks mandatory for public transport in Greater Sydney'](https://www.abc.net.au/news/2021-06-18/nsw-records-one-covid-19-case-masks-for-public-transport/100225370)

In [ ]:
pd.read_sql_query("""WITH lga_top10_all AS (
                         SELECT lga,
                             COUNT(*) 'cases'
                         FROM covid_nsw_2
                         GROUP BY 1
                         ORDER BY 2 DESC
                         LIMIT 10
                     ),
                     
                     cases_pre_delta AS (
                         SELECT lga,
                             COUNT(*) 'cases_pre_delta'
                         FROM covid_nsw_2
                         WHERE notification_date < '2021-06-18'
                             AND lga IN
                                 (SELECT lga
                                  FROM lga_top10_all
                                 )
                         GROUP BY 1
                         ORDER BY 2 DESC
                     )
                     
                     SELECT l.lga,
                         c.cases_pre_delta,
                         (l.cases - c.cases_pre_delta) 'cases_post_delta',
                         l.cases 'cases_total'
                     FROM lga_top10_all l
                     LEFT JOIN cases_pre_delta c
                         ON l.lga = c.lga
                     ORDER BY 4 DESC""", engine)

##### No reported cases

In [ ]:
pd.read_sql_query("""SELECT DISTINCT lga
                     FROM postcode_to_lga
                     WHERE lga NOT IN (
                         SELECT lga
                         FROM covid_nsw_2
                         )
                     ORDER BY 1""", engine)